In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField , StructType , StringType , IntegerType , ArrayType , LongType
from pyspark.sql.functions import from_json , col , explode



In [2]:
spark = (
    SparkSession.builder
    .appName("Streaming from Kafka")
    .config("spark.streaming.stopGracefullyOnShutdown", "true")
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.1")
    .config("spark.jars" , '/home/mhai/test/postgresql-42.7.3.jar')
    .config("spark.sql.shuffle.partitions", "5")
    .master("local[*]")
    .getOrCreate()
)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/28 22:05:28 WARN Utils: Your hostname, Admin-PC, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/28 22:05:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/mhai/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/mhai/.ivy2.5.2/cache
The jars for the packages stored in: /home/mhai/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-22add1c0-3b93-4179-a848-75407d0bca43;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.s

In [3]:
spark
    


In [3]:
print(spark.sparkContext._jsc.sc().listJars())


List(spark://10.255.255.254:41785/jars/org.apache.spark_spark-sql-kafka-0-10_2.13-4.0.1.jar, spark://10.255.255.254:41785/jars/org.apache.hadoop_hadoop-client-runtime-3.4.1.jar, spark://10.255.255.254:41785/jars/com.google.code.findbugs_jsr305-3.0.0.jar, spark://10.255.255.254:41785/jars/org.apache.commons_commons-pool2-2.12.0.jar, spark://10.255.255.254:41785/jars/org.xerial.snappy_snappy-java-1.1.10.7.jar, spark://10.255.255.254:41785/jars/org.slf4j_slf4j-api-2.0.16.jar, spark://10.255.255.254:41785/jars/org.apache.kafka_kafka-clients-3.9.1.jar, spark://10.255.255.254:41785/jars/org.lz4_lz4-java-1.8.0.jar, spark://10.255.255.254:41785/jars/org.apache.spark_spark-token-provider-kafka-0-10_2.13-4.0.1.jar, spark://10.255.255.254:41785/jars/org.scala-lang.modules_scala-parallel-collections_2.13-1.2.0.jar, spark://10.255.255.254:41785/jars/org.apache.hadoop_hadoop-client-api-3.4.1.jar)


In [63]:
device = spark.readStream.format('kafka') \
                   .option('kafka.bootstrap.servers' , '172.23.152.231:9092') \
                   .option('subscribe' , 'device-data') \
                   .option('startingOffsets' , 'earliest') \
                   .load()

In [64]:
device.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [65]:
device = device.withColumn('value' , device['value'].cast('string'))

In [66]:
device.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [67]:
device_schema = StructType(
    [
        StructField('customerId' , StringType() , True) ,
        StructField('data' , StructType(
            [StructField('devices' ,
                ArrayType(StructType([
                    StructField('deviceId' , StringType() , True) ,
                    StructField('measure' , StringType() , True) ,
                    StructField('status' , StringType() , True) ,
                    StructField('temperature' , StringType() , True)
                ]),True ) , True ) 
            ]) , True) ,
        StructField('eventId' , StringType() , True) ,
        StructField('eventOffset' , LongType() , True) ,
        StructField('eventPublisher' , StringType() , True) ,
        StructField('eventTime' , StringType(), True )          
    ]
)

In [68]:
device = device.withColumn('value_json' , from_json(col('value') , device_schema)).select('value_json.*')

In [69]:
device_explode = device.withColumn('devices' , explode(col('data.devices')))

In [70]:
device = device_explode.withColumn('deviceId', col('devices.deviceId')) \
                        .withColumn('measure', col('devices.measure')) \
                        .withColumn('status', col('devices.status')) \
                        .withColumn('temperature', col('devices.temperature')) \
                        .drop('data') \
                        .drop('devices')

In [71]:
def save_psql(data , batch_id) :
    print('batch id : ' , str(batch_id))    
    data.write.format('jdbc') \
            .mode('append') \
            .option('driver' , 'org.postgresql.Driver') \
            .option('url' , 'jdbc:postgresql://192.168.1.5:5432/test') \
            .option('dbtable' , 'public.device_data') \
            .option('user' , 'postgres') \
            .option('password' , 'MINHHAI123') \
            .save()
    data.show()
    

In [73]:
device.printSchema()

root
 |-- customerId: string (nullable = true)
 |-- eventId: string (nullable = true)
 |-- eventOffset: long (nullable = true)
 |-- eventPublisher: string (nullable = true)
 |-- eventTime: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- measure: string (nullable = true)
 |-- status: string (nullable = true)
 |-- temperature: string (nullable = true)



In [ ]:
device.writeStream \
        .foreachBatch(save_psql) \
        .outputMode('append') \
        .trigger(processingTime = '10 seconds') \
        .option('checkpointLocation' , 'checkpoint_dir_kafka') \
        .start() \
        .awaitTermination() 

batch id :  0


25/10/28 22:42:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00103|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|    D001|      C|   ERROR|         15|
|   CI00103|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|    D002|      C| SUCCESS|         16|
|   CI00101|1450324a-c546-417...|      10038|        device|2023-01-05 11:13:...|    D004|      C| SUCCESS|         20|
|   CI00101|1450324a-c546-417...|      10038|        device|2023-01-05 11:13:...|    D004|      C| SUCCESS|          1|
|   CI00101|1450324a-c546-417...|      10038|        device|2023-01-05 11:13:...|    D002|      C| SUCCESS|         21|
|   CI00108|aa90011f-3967-496...|      1

+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00217|5ae70448-0459-4c0...|      10000|        device|2025-10-28 22:45:...|       v|      C|STRANDBY|         30|
|   CI00217|5ae70448-0459-4c0...|      10000|        device|2025-10-28 22:45:...|       e|      C| SUCCESS|         49|
|   CI00217|5ae70448-0459-4c0...|      10000|        device|2025-10-28 22:45:...|       c|      C|    NULL|         40|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+

batch id :  2
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             

+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure| status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+
|   CI00298|9189c37c-2633-40c...|      10007|        device|2025-10-28 22:45:...|       c|      C|   NULL|         24|
|   CI00298|9189c37c-2633-40c...|      10007|        device|2025-10-28 22:45:...|       e|      C|SUCCESS|          4|
|   CI00298|9189c37c-2633-40c...|      10007|        device|2025-10-28 22:45:...|       v|      C|  ERROR|         23|
|   CI00238|c698ea95-e282-45b...|      10008|        device|2025-10-28 22:45:...|       i|      C|   NULL|          4|
|   CI00238|c698ea95-e282-45b...|      10008|        device|2025-10-28 22:45:...|       I|      C|  ERROR|         48|
|   CI00238|c698ea95-e282-45b...|      10008|   

...

+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure| status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+-------+-----------+
|   CI00206|6488c795-ffb7-4c8...|      10145|        device|2025-10-28 22:54:...|       i|      C|   NULL|          9|
|   CI00206|6488c795-ffb7-4c8...|      10145|        device|2025-10-28 22:54:...|       _|      C|SUCCESS|         20|
|   CI00291|abc04f82-b8fd-40b...|      10146|        device|2025-10-28 22:54:...|       c|      C|   NULL|          6|
|   CI00196|93dfc987-fcab-4d5...|      10147|        device|2025-10-28 22:54:...|       e|      C|   NULL|         18|
|   CI00196|93dfc987-fcab-4d5...|      10147|        device|2025-10-28 22:54:...|       e|      C|  ERROR|         46|
|   CI00196|93dfc987-fcab-4d5...|      10147|   

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/mhai/test/venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mhai/test/venv/lib/python3.12/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|customerId|             eventId|eventOffset|eventPublisher|           eventTime|deviceId|measure|  status|temperature|
+----------+--------------------+-----------+--------------+--------------------+--------+-------+--------+-----------+
|   CI00220|25a98c94-574b-48f...|      10148|        device|2025-10-28 22:54:...|       e|      C| SUCCESS|         32|
|   CI00220|25a98c94-574b-48f...|      10148|        device|2025-10-28 22:54:...|       I|      C| SUCCESS|          4|
|   CI00107|b23dc482-1fb5-42d...|      10149|        device|2025-10-28 22:54:...|       I|      C| SUCCESS|         28|
|   CI00107|b23dc482-1fb5-42d...|      10149|        device|2025-10-28 22:54:...|       e|      C| SUCCESS|         24|
|   CI00107|b23dc482-1fb5-42d...|      10149|        device|2025-10-28 22:54:...|       e|      C| SUCCESS|         28|
|   CI00107|b23dc482-1fb5-42d...|      1

KeyboardInterrupt: 

batch id :  55
